In [14]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

import logging
import ipywidgets as widgets
from pprint import pprint

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

from haystack.nodes import FARMReader
from haystack.nodes import TfidfRetriever
from haystack.pipelines import ExtractiveQAPipeline
from haystack.document_stores import InMemoryDocumentStore
from haystack.utils import clean_wiki_text, convert_files_to_docs

In [30]:
document_store = InMemoryDocumentStore()

doc_dir = f"{os.getcwd()}/data/text"

docs = convert_files_to_docs(dir_path=doc_dir)
document_store.write_documents(docs)

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.utils.preprocessing -  Converting /data/user/home/jesusaur/cs662-qa-land-dev-law-sys/programs/data/text/text_2.txt
INFO - haystack.utils.preprocessing -  Converting /data/user/home/jesusaur/cs662-qa-land-dev-law-sys/programs/data/text/text_3.txt
INFO - haystack.utils.preprocessing -  Converting /data/user/home/jesusaur/cs662-qa-land-dev-law-sys/programs/data/text/text_11.txt
INFO - haystack.utils.preprocessing -  Converting /data/user/home/jesusaur/cs662-qa-land-dev-law-sys/programs/data/text/text_12.txt
INFO - haystack.utils.preprocessing -  Converting /data/user/home/jesusaur/cs662-qa-land-dev-law-sys/programs/data/text/text_5.txt
INFO - haystack.utils.preprocessing -  Converting /data/user/home/jesusaur/cs662-qa-land-dev-law-sys/programs/data/text/text_7.txt
INFO - haystack.utils.preprocessing -  Converting /data/user/home/jesusaur/cs662-qa-land-dev-law-sys/programs/data/text/text_1.txt
INFO 

In [31]:
retriever = TfidfRetriever(document_store=document_store)

INFO - haystack.nodes.retriever.sparse -  Found 13 candidate paragraphs from 13 docs in DB


In [32]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO - haystack.modeling.model.language_model -  Auto-detected model language: english
INFO - haystack.modeling.model.language_model -  Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


In [33]:
pipe = ExtractiveQAPipeline(reader, retriever)

In [25]:
txtsl = widgets.Text( # 'Which zoning districts allow group care facilities?'
 placeholder='Enter your question.',
 description='Question:'
 )
display(txtsl)

Text(value='', description='Question:', placeholder='Enter your question.')

In [38]:
prediction = pipe.run(
    query=txtsl.value, params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 1}}
)

Inferencing Samples: 100%|██████████| 526/526 [03:28<00:00,  2.52 Batches/s]
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-19175, -19128) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-3134, -3109) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-3393, -3322) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-24641, -24600) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-5241, -5208) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-15063, -14986) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-9213, -9191) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-8147, -8124) with a span answer. 
ERROR - haystack.modeling.model.predictions -  Invalid end offset: 
(-28818, -28636) 

In [39]:
pprint(prediction['answers'])

[<Answer {'answer': 'to implement further the following pol', 'type': 'extractive', 'score': 0.909953773021698, 'context': 'the rural landscape. The intent of the district also is to implement further the following policies of the Comprehensive Plan: To further identify sce', 'offsets_in_document': [{'start': 14827, 'end': 14865}], 'offsets_in_context': [{'start': 56, 'end': 94}], 'document_id': 'ba462978e0dc0fba8a7a151b82d64e78', 'meta': {'name': 'text_8.txt'}}>]
